In [1]:
!pip install soundfile ftfy regex einops scipy tqdm librosa timm opencv-python-headless ffmpeg-python py7zr



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 132.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.5/791.5 kB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 86.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install --upgrade pytorchvideo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 81.3 MB/s eta 0:00:00:00:0100:01
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188685 sha256=adc6b1ed452f0f0853977c19b9a01f2572d87f67a28bc7d1eae624620929d099
  Stored in directory: /root/.cache/pip/wheels/ff/4e/81/0f72a543be9ed7eb737c95bfc5da4025e73226b44368074ece
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=eb739044fcc50ba3f3fe1a43f263ceb08dd32b6c3bdd423fae7b1fcff07e301d
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Crea

In [13]:
import os
import re
import json

root_dir = "/workspace/dataset/S5"
txt_dir = os.path.join(root_dir, "TXT")


pattern = re.compile(r"(\S+)\s*\[(\d+\.\d+)-(\d+\.\d+)\]:\s*(.*)")

dataset = []

for fname in os.listdir(txt_dir):
    if not fname.endswith(".txt"):
        continue

    video_name = os.path.splitext(fname)[0]
    txt_path = os.path.join(txt_dir, fname)
    video_dir = os.path.join(root_dir, video_name)

    if not os.path.exists(video_dir):
        print(f"[WARN] missing video folder for {video_name}")
        continue

    with open(txt_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    for line in lines:
        line = line.strip()
        m = pattern.match(line)
        if not m:
            continue

        utt_id, start, end, text = m.groups()
        video_path = os.path.join(video_dir, f"{utt_id}.mp4")
        audio_path = os.path.join(video_dir, f"{utt_id}.wav")

        if not (os.path.exists(video_path) and os.path.exists(audio_path)):
            print(f"[WARN] missing segment for {utt_id}")
            continue

        dataset.append({
            "utt_id": utt_id,
            "video_path": video_path,
            "audio_path": audio_path,
            "text": text.strip(),
            "start_time": float(start),
            "end_time": float(end)
        })

print(f"Parsed {len(dataset)} utterances in total.")
os.makedirs(os.path.join(root_dir, "meta"), exist_ok=True)

out_json = os.path.join(root_dir, "meta", "train_meta.json")
with open(out_json, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=2)

print(f" Metadata saved to: {out_json}")


[WARN] missing segment for Ses05M_impro06_FXX2
Parsed 2195 utterances in total.
 Metadata saved to: /workspace/dataset/S5/meta/train_meta.json


In [6]:
!pip uninstall -y torch torchvision torchaudio pytorchvideo transformers accelerate

!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 pytorchvideo==0.1.5




Found existing installation: torch 2.4.0+cu121
Uninstalling torch-2.4.0+cu121:
  Successfully uninstalled torch-2.4.0+cu121
Found existing installation: torchvision 0.19.0+cu121
Uninstalling torchvision-0.19.0+cu121:
  Successfully uninstalled torchvision-0.19.0+cu121
Found existing installation: torchaudio 2.4.0+cu121
Uninstalling torchaudio-2.4.0+cu121:
  Successfully uninstalled torchaudio-2.4.0+cu121
Found existing installation: pytorchvideo 0.1.5
Uninstalling pytorchvideo-0.1.5:
  Successfully uninstalled pytorchvideo-0.1.5
  Using cached torch-2.1.2-cp310-cp310-manylinux1_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.16.2-cp310-cp310-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached torchaudio-2.1.2-cp310-cp310-manylinux1_x86_64.whl.metadata (6.4 kB)
  Using cached pytorchvideo-0.1.5-py3-none-any.whl
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.18.1-py3-none-manylinux1_x86_64.whl.metad

In [22]:
import torch, torchvision, torchaudio; print(torch.__version__, torchvision.__version__, torchaudio.__version__)

2.1.2+cu121 0.16.2+cu121 2.1.2+cu121


In [23]:
'''
 image bind 的定义
def load_and_transform_vision_data(image_paths, device):
    if image_paths is None:
        return None

    image_outputs = []

    data_transform = transforms.Compose(
        [
            transforms.Resize(
                224, interpolation=transforms.InterpolationMode.BICUBIC
            ),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=(0.48145466, 0.4578275, 0.40821073),
                std=(0.26862954, 0.26130258, 0.27577711),
            ),
        ]
    )
    
    for image_path in image_paths:
        with open(image_path, "rb") as fopen:
            image = Image.open(fopen).convert("RGB")

        image = data_transform(image).to(device)
        image_outputs.append(image)
    return torch.stack(image_outputs, dim=0)
    
'''

'\n image bind 的定义\ndef load_and_transform_vision_data(image_paths, device):\n    if image_paths is None:\n        return None\n\n    image_outputs = []\n\n    data_transform = transforms.Compose(\n        [\n            transforms.Resize(\n                224, interpolation=transforms.InterpolationMode.BICUBIC\n            ),\n            transforms.CenterCrop(224),\n            transforms.ToTensor(),\n            transforms.Normalize(\n                mean=(0.48145466, 0.4578275, 0.40821073),\n                std=(0.26862954, 0.26130258, 0.27577711),\n            ),\n        ]\n    )\n    \n    for image_path in image_paths:\n        with open(image_path, "rb") as fopen:\n            image = Image.open(fopen).convert("RGB")\n\n        image = data_transform(image).to(device)\n        image_outputs.append(image)\n    return torch.stack(image_outputs, dim=0)\n    \n'

In [9]:
!pip install decord


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 96.8 MB/s eta 0:00:00ta 0:00:01


In [15]:
!pip install numpy


In [5]:
torch.version.cuda

'12.1'

In [2]:
pip install numpy==1.26.4 --force-reinstall


  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch, numpy as np, torchvision
print("Torch:", torch.__version__)
print("Torchvision:", torchvision.__version__)
print("NumPy:", np.__version__)

x = torch.rand(3,3)
print("Tensor to numpy:", x.numpy())


Torch: 2.1.2+cu121
Torchvision: 0.16.2+cu121
NumPy: 1.26.4
Tensor to numpy: [[0.15420312 0.39508873 0.0564419 ]
 [0.42654216 0.29004008 0.9550649 ]
 [0.37730187 0.04210186 0.43736172]]


In [2]:
import torchvision
import os
print(os.path.exists(os.path.join(os.path.dirname(torchvision.__file__), "transforms", "functional_tensor.py")))


True


In [ ]:
pip install torchvision==0.17.2 --force-reinstall


In [7]:
pip install --force-reinstall torchvision==0.19.0 --extra-index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.19.0%2Bcu121-cp310-cp310-linux_x86_64.whl (7.1 MB)
  Using cached numpy-2.2.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached https://download.pytorch.org/whl/cu121/torch-2.4.0%2Bcu121-cp310-cp310-linux_x86_64.whl (799.1 MB)
  Using cached https://download.pytorch.org/whl/sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached https://download.pytorch.org/whl/cu121/nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014

In [1]:
import os
import json
import torch
import math
import numpy as np

from pytorchvideo.data.encoded_video import EncodedVideo
from torchvision.transforms import Compose, Lambda
from pytorchvideo.transforms import UniformTemporalSubsample
from imagebind import data
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType
import torchvision.transforms as T
device = "cuda:0"
model = imagebind_model.imagebind_huge(pretrained=True)
for param in model.parameters():
    param.requires_grad = False 
model.eval().to(device)

meta_path = "/workspace/dataset/S1/meta/train_meta.json"
save_dir = "/workspace/dataset/S1/features"
os.makedirs(save_dir, exist_ok=True)


def load_and_transform_vision_data_from_tensor(video_tensor, device):
    """
    模仿 imagebind.data.load_and_transform_vision_data 的行为，
    对视频帧张量做标准化预处理后送入模型。
    video_tensor: torch.Tensor [C, T, H, W]
    """
    # ImageBind vision 模块的标准归一化参数
    mean = torch.tensor([0.48145466, 0.4578275, 0.40821073], device=device)[:, None, None, None]
    std = torch.tensor([0.26862954, 0.26130258, 0.27577711], device=device)[:, None, None, None]

    # Resize + CenterCrop（保持一致性）
    transform = T.Compose([
        T.Resize((224, 224)),
    ])

    # video_tensor: (C, T, H, W)
    frames = []
    for t in range(video_tensor.shape[1]):
        frame = video_tensor[:, t, :, :].cpu()
        frame = T.ToPILImage()(frame)
        frame = transform(frame)
        frame = T.ToTensor()(frame)
        frames.append(frame)

    x = torch.stack(frames, dim=0).to(device)  # [T, 3, 224, 224]
    x = (x - mean.permute(1, 0, 2, 3)) / std.permute(1, 0, 2, 3)  # 归一化
    return x

"""
def load_video_gpu(video_path, num_frames=16, max_duration=15.0):
    
    #用 PyTorchVideo 直接在 GPU 上读取视频并采样为若干帧（不落地jpg）
    
    video = EncodedVideo.from_path(video_path)
    duration = video.duration

    # 截断长视频
    end_sec = min(duration, max_duration)
    clip = video.get_clip(start_sec=0, end_sec=end_sec)

    video_data = clip["video"]  # (C, T, H, W)
    T = video_data.shape[1]
    if T > num_frames:
        sampler = UniformTemporalSubsample(num_frames)
        video_data = sampler(video_data)
    elif T < num_frames:
        repeat = (num_frames // T) + 1
        video_data = video_data.repeat(1, repeat, 1, 1)[:, :num_frames]

    return video_data  # (3, num_frames, H, W)
"""

def load_video_gpu(video_path, fps=8, max_duration=15.0):
    # 用 decord，只解视频帧，不读音频
    video = EncodedVideo.from_path(video_path, decoder="decord", decode_audio=False)

    duration = getattr(video, "duration", None)
    if duration is None or duration <= 0:
        print(f"[WARN] Invalid duration for {video_path}, skipping.")
        return None

    duration = min(duration, max_duration)
    clip = video.get_clip(start_sec=0, end_sec=duration)

    if clip is None or clip.get("video") is None:
        print(f"[WARN] Failed to read video from {video_path}")
        return None

    video_data = clip["video"]
    if video_data is None:
        print(f"[WARN] Empty video data for {video_path}")
        return None

    # 目标帧数 = fps * duration
    target_frames = int(math.ceil(fps * duration))
    T = video_data.shape[1]

    if T > target_frames:
        indices = torch.linspace(0, T - 1, target_frames).long()
        video_data = video_data[:, indices, :, :]
    elif T < target_frames and T > 0:
        repeat = (target_frames // T) + 1
        video_data = video_data.repeat(1, repeat, 1, 1)[:, :target_frames]

    return video_data



def extract_embeddings(meta_file):
    with open(meta_file, "r", encoding="utf-8") as f:
        items = json.load(f)

    grouped = {}
    for item in items:
        vid = item["video_path"].split("/")[-2]
        grouped.setdefault(vid, []).append(item)

    for vid, utterances in grouped.items():
        print(f" Processing {vid} ...")
        all_embeddings = {}

        for item in utterances:
            utt_id = item["utt_id"]
            text = item["text"]
            audio_path = item["audio_path"]
            video_path = item["video_path"]

            # 加载起止时间（可选字段）
            start_time = item.get("start_time", None)
            end_time = item.get("end_time", None)

            # 加载视频
            video_tensor = load_video_gpu(video_path)
            vision_input = load_and_transform_vision_data_from_tensor(video_tensor, device)

            # 文本 & 音频
            text_input = data.load_and_transform_text([text], device)
            audio_input = data.load_and_transform_audio_data([audio_path], device)

            inputs = {
                ModalityType.TEXT: text_input,
                ModalityType.AUDIO: audio_input,
                ModalityType.VISION: vision_input,
            }

            with torch.no_grad():
                emb = model(inputs)

            #  保存 embedding + 时间信息
            all_embeddings[utt_id] = {
                "text_emb": emb[ModalityType.TEXT].cpu(),
                "audio_emb": emb[ModalityType.AUDIO].cpu(),
                "vision_emb": emb[ModalityType.VISION].mean(dim=0, keepdim=True).cpu(),
                "start_time": start_time,
                "end_time": end_time,
            }

        # 保存当前视频的所有 embedding
        save_path = os.path.join(save_dir, f"{vid}.pt")
        torch.save(all_embeddings, save_path)
        print(f" Saved {save_path} ({len(all_embeddings)} utterances)")

    print(" All videos processed!")



if __name__ == "__main__":
    extract_embeddings(meta_path)


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/timm/mode

 Processing Ses01F_impro03 ...
 Saved /workspace/dataset/S1/features/Ses01F_impro03.pt (52 utterances)
 Processing Ses01F_impro01 ...
 Saved /workspace/dataset/S1/features/Ses01F_impro01.pt (30 utterances)
 Processing Ses01F_impro02 ...
 Saved /workspace/dataset/S1/features/Ses01F_impro02.pt (38 utterances)
 All videos processed!


In [12]:
import torch
import os

feature_dir = "/workspace/dataset/S111/features"

# 读取某个 session 的 embeddings
path = os.path.join(feature_dir, "Ses01F_impro01.pt")
data = torch.load(path)

print(len(data))  # 一共有多少个utterance
print(list(data.keys())[:3])  # 看前三个id

utt_id = "Ses01F_impro01_F000"
print(data[utt_id]["text_emb"].shape)
print(data[utt_id]["vision_emb"].shape)

30
['Ses01F_impro01_F000', 'Ses01F_impro01_M000', 'Ses01F_impro01_F001']
torch.Size([1, 1024])
torch.Size([1, 1024])


In [14]:
import os
import json
import torch
import math
import numpy as np

from pytorchvideo.data.encoded_video import EncodedVideo
from torchvision.transforms import Compose, Lambda
from pytorchvideo.transforms import UniformTemporalSubsample
from imagebind import data
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType
import torchvision.transforms as T

# ===== 初始化模型 =====
device = "cuda:0"
model = imagebind_model.imagebind_huge(pretrained=True)
for param in model.parameters():
    param.requires_grad = False 
model.eval().to(device)


# ====== 视频预处理函数 ======
def load_and_transform_vision_data_from_tensor(video_tensor, device):
    mean = torch.tensor([0.48145466, 0.4578275, 0.40821073], device=device)[:, None, None, None]
    std = torch.tensor([0.26862954, 0.26130258, 0.27577711], device=device)[:, None, None, None]

    transform = T.Compose([T.Resize((224, 224))])
    frames = []
    for t in range(video_tensor.shape[1]):
        frame = video_tensor[:, t, :, :].cpu()
        frame = T.ToPILImage()(frame)
        frame = transform(frame)
        frame = T.ToTensor()(frame)
        frames.append(frame)

    x = torch.stack(frames, dim=0).to(device)
    x = (x - mean.permute(1, 0, 2, 3)) / std.permute(1, 0, 2, 3)
    return x


def load_video_gpu(video_path, fps=8, max_duration=15.0):
    video = EncodedVideo.from_path(video_path, decoder="decord", decode_audio=False)
    duration = getattr(video, "duration", None)
    if duration is None or duration <= 0:
        print(f"[WARN] Invalid duration for {video_path}, skipping.")
        return None

    duration = min(duration, max_duration)
    clip = video.get_clip(start_sec=0, end_sec=duration)
    if clip is None or clip.get("video") is None:
        print(f"[WARN] Failed to read video from {video_path}")
        return None

    video_data = clip["video"]
    if video_data is None:
        print(f"[WARN] Empty video data for {video_path}")
        return None

    target_frames = int(math.ceil(fps * duration))
    T = video_data.shape[1]
    if T > target_frames:
        indices = torch.linspace(0, T - 1, target_frames).long()
        video_data = video_data[:, indices, :, :]
    elif T < target_frames and T > 0:
        repeat = (target_frames // T) + 1
        video_data = video_data.repeat(1, repeat, 1, 1)[:, :target_frames]
    return video_data


# ====== 提取 Embedding 函数 ======
def extract_embeddings(meta_file, save_dir):
    os.makedirs(save_dir, exist_ok=True)

    with open(meta_file, "r", encoding="utf-8") as f:
        items = json.load(f)

    grouped = {}
    for item in items:
        vid = item["video_path"].split("/")[-2]
        grouped.setdefault(vid, []).append(item)

    for vid, utterances in grouped.items():
        print(f" Processing {vid} ...")
        all_embeddings = {}

        for item in utterances:
            utt_id = item["utt_id"]
            text = item["text"]
            audio_path = item["audio_path"]
            video_path = item["video_path"]

            start_time = item.get("start_time", None)
            end_time = item.get("end_time", None)

            video_tensor = load_video_gpu(video_path)
            vision_input = load_and_transform_vision_data_from_tensor(video_tensor, device)
            text_input = data.load_and_transform_text([text], device)
            audio_input = data.load_and_transform_audio_data([audio_path], device)

            inputs = {
                ModalityType.TEXT: text_input,
                ModalityType.AUDIO: audio_input,
                ModalityType.VISION: vision_input,
            }

            with torch.no_grad():
                emb = model(inputs)

            all_embeddings[utt_id] = {
                "text_emb": emb[ModalityType.TEXT].cpu(),
                "audio_emb": emb[ModalityType.AUDIO].cpu(),
                "vision_emb": emb[ModalityType.VISION].mean(dim=0, keepdim=True).cpu(),
                "start_time": start_time,
                "end_time": end_time,
                "text": text,
            }

        save_path = os.path.join(save_dir, f"{vid}.pt")
        torch.save(all_embeddings, save_path)
        print(f"Saved {save_path} ({len(all_embeddings)} utterances)")

    print("All videos processed for this session.")


# ====== 批量处理 S1–S5 ======
if __name__ == "__main__":
    base_dir = "/workspace/dataset"
    sessions = ["S1", "S2", "S3", "S4", "S5"]

    for s in sessions:
        meta_path = os.path.join(base_dir, s, "meta", "train_meta.json")
        save_dir = os.path.join(base_dir, s, "features")

        if not os.path.exists(meta_path):
            print(f"[WARN] {meta_path} not found, skipping.")
            continue

        print(f"\n=== Processing {s} ===")
        extract_embeddings(meta_path, save_dir)



=== Processing S1 ===
 Processing Ses01F_impro01 ...
Saved /workspace/dataset/S1/features/Ses01F_impro01.pt (30 utterances)
 Processing Ses01F_impro02 ...
Saved /workspace/dataset/S1/features/Ses01F_impro02.pt (38 utterances)
 Processing Ses01F_impro03 ...
Saved /workspace/dataset/S1/features/Ses01F_impro03.pt (52 utterances)
 Processing Ses01F_impro04 ...
Saved /workspace/dataset/S1/features/Ses01F_impro04.pt (71 utterances)
 Processing Ses01F_impro05 ...
Saved /workspace/dataset/S1/features/Ses01F_impro05.pt (67 utterances)
 Processing Ses01F_impro06 ...
Saved /workspace/dataset/S1/features/Ses01F_impro06.pt (47 utterances)
 Processing Ses01F_impro07 ...
Saved /workspace/dataset/S1/features/Ses01F_impro07.pt (37 utterances)
 Processing Ses01F_script01_1 ...
Saved /workspace/dataset/S1/features/Ses01F_script01_1.pt (89 utterances)
 Processing Ses01F_script01_2 ...
Saved /workspace/dataset/S1/features/Ses01F_script01_2.pt (33 utterances)
 Processing Ses01F_script01_3 ...
Saved /worksp

In [15]:
import os
import re
import torch

def parse_large_txt_strict(txt_path):
    """
    严格解析 .txt 文件，确保 ID 一一对应
    返回 dict: {utt_id: {...info...}}
    """
    with open(txt_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    data = {}
    current_id = None

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # 识别 utterance 的头行
        header = re.match(r"\[(.*?) - (.*?)\]\s+(\S+)\s+\S+\s+\[(.*?)\]", line)
        if header:
            start, end, utt_id, vad_str = header.groups()
            vad = [float(x.strip()) for x in vad_str.split(",")]
            data[utt_id] = {
                "start_time": float(start),
                "end_time": float(end),
                "VAD": vad,
                "C-E1": None,
                "C-E2": None,
                "C-E4": None,
            }
            current_id = utt_id
            continue

        # 在 block 内读取 C-E1/C-E2/C-E4
        if current_id:
            for tag in ["C-E1", "C-E2", "C-E4"]:
                if line.startswith(f"{tag}:"):
                    emo = line.split(":")[1].split(";")[0].strip()
                    data[current_id][tag] = emo
    return data


def merge_with_pt_strict(pt_path, txt_path):
    """
    严格根据 utterance ID 对齐并合并标注信息
    """
    print(f"Merging {os.path.basename(pt_path)}")

    info_dict = parse_large_txt_strict(txt_path)
    data = torch.load(pt_path, map_location="cpu")

    matched, missed = 0, 0
    for utt_id, sample in data.items():
        if utt_id in info_dict:
            sample.update(info_dict[utt_id])
            matched += 1
        else:
            missed += 1
            print(f"No label for {utt_id}")

    save_path = pt_path.replace(".pt", "_with_emo.pt")
    torch.save(data, save_path)
    print(f"Saved: {save_path} (Matched {matched}, Missed {missed})\n")


def batch_process(base_dir="/workspace/dataset"):
    for s in ["S1", "S2", "S3", "S4", "S5"]:
        feat_dir = os.path.join(base_dir, s, "features")
        txt_dir = os.path.join(base_dir, s, "LargeWords")

        if not os.path.exists(feat_dir) or not os.path.exists(txt_dir):
            continue

        for fname in os.listdir(feat_dir):
            if not fname.endswith(".pt"):
                continue
            pt_path = os.path.join(feat_dir, fname)
            base_name = fname.replace(".pt", "")
            txt_path = os.path.join(txt_dir, f"{base_name}.txt")

            if os.path.exists(txt_path):
                merge_with_pt_strict(pt_path, txt_path)
            else:
                print(f"Missing txt: {txt_path}")


if __name__ == "__main__":
    batch_process("/workspace/dataset")



Merging Ses01F_impro01.pt
Saved: /workspace/dataset/S1/features/Ses01F_impro01_with_emo.pt (Matched 30, Missed 0)

Merging Ses01F_impro02.pt
Saved: /workspace/dataset/S1/features/Ses01F_impro02_with_emo.pt (Matched 38, Missed 0)

Merging Ses01F_impro03.pt
Saved: /workspace/dataset/S1/features/Ses01F_impro03_with_emo.pt (Matched 52, Missed 0)

Merging Ses01F_impro04.pt
Saved: /workspace/dataset/S1/features/Ses01F_impro04_with_emo.pt (Matched 71, Missed 0)

Merging Ses01F_impro05.pt
Saved: /workspace/dataset/S1/features/Ses01F_impro05_with_emo.pt (Matched 67, Missed 0)

Merging Ses01F_impro06.pt
Saved: /workspace/dataset/S1/features/Ses01F_impro06_with_emo.pt (Matched 47, Missed 0)

Merging Ses01F_impro07.pt
Saved: /workspace/dataset/S1/features/Ses01F_impro07_with_emo.pt (Matched 37, Missed 0)

Merging Ses01F_script01_1.pt
Saved: /workspace/dataset/S1/features/Ses01F_script01_1_with_emo.pt (Matched 89, Missed 0)

Merging Ses01F_script01_2.pt
Saved: /workspace/dataset/S1/features/Ses01F_